In [1]:
!pip install gensim==3.8.3 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 24.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: gensim
    Found existing installation: gensim 4.0.1
    Uninstalling gensim-4.0.1:
      Successfully uninstalled gensim-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scattertext 0.1.7 requires gensim>=4.0.0, but you have gensim 3.8.3 which is incompatible.


In [2]:
import pandas as pd
from nltk.tokenize import sent_tokenize
from wordcloud import WordCloud, ImageColorGenerator
from gensim.summarization.textcleaner import split_sentences
import nltk
import string
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
train_set = pd.read_excel("../input/summerization/Competition CERIST -- Summarization/Dataset 2 Arabic + English (XL_sum)/English/dataset_XL_sum_v1.0_train_en.xlsx")

In [4]:
train_set

,Document,Summary
0,Atlantis Resources unveiled the marine energy ...,The massive tidal turbine AK1000 has been inst...
1,Police were called to the scene outside the Co...,A man has been stabbed in broad daylight outsi...
2,Anthony ZurcherNorth America reporter@awzurche...,It was inevitable that the fallout from the US...
3,By Debbie JacksonBBC Scotland But while most o...,Week four of social distancing is starting to ...
4,He was arrested on Saturday morning and is cur...,A 37-year-old man has been arrested as part an...
...,...,...
304763,He said it made him sad when many phones were ...,Pope Francis has chided the Catholic faithful ...
304764,By Flora Carmichael and Marianna SpringBBC Tre...,Coronavirus misinformation is flooding the int...
304765,By Flora Carmichael and Marianna SpringBBC Tre...,Coronavirus misinformation is flooding the int...
304766,The airline has operated a daily service to Ne...,American Airlines is pulling out of Birmingham...


In [5]:
train_set.describe()

,Document,Summary
count,304768,304768
unique,304739,304112
top,Please also ensure you follow current coronavi...,With the world's biggest bike race starting in...
freq,19,30


In [6]:
# the number of sentences in the Summary
train_set['sum_sent_count'] = train_set['Summary'].apply(lambda x : len(split_sentences(x)))
#  the number of sentences in the text
train_set['text_sent_count'] = train_set['Document'].apply(lambda x : len(split_sentences(x)))

In [7]:
# the number of word in the summary
train_set['sum_word_count'] = train_set['Summary'].apply(lambda x : len(x.split()))
# the number of word in the text
train_set['text_word_count'] = train_set['Document'].apply(lambda x : len(x.split()))

In [8]:

# the number of characters in the summary
train_set['sum_char_count'] = train_set['Summary'].apply(lambda x : len(x.replace(" ","")))
#the number of characters in the text
train_set['text_char_count'] = train_set['Document'].apply(lambda x : len(x.replace(" ","")))

In [9]:
# the density of words in the summary
train_set['sum_word_density'] = train_set['sum_word_count'] / (train_set['sum_char_count'] + 1)
#  the density of words in the text
train_set['text_word_density'] = train_set['text_word_count'] / (train_set['text_char_count'] + 1)

In [10]:
# the density of sentences in the summary
train_set['sum_sent_density'] = train_set['sum_sent_count'] / (train_set['sum_word_count'] + 1)
# the density of sentences in the text
train_set['text_sent_density'] = train_set['text_sent_count'] / (train_set['text_word_count'] + 1)

In [11]:
def count_stopwords(text):
    ''' Return the number of stopwords in the text
        Input:
            - text: string
            - stopwords: list of string, containing the stopwords
        Output:
            - int, number of stopwords in the text argument
    '''
    word_tokens =nltk.word_tokenize(text)
    stopwords_x = [w for w in word_tokens if w in  set(stopwords.words('english'))]
    
    return len(stopwords_x)

In [12]:

# the number of punctuations in the summary
train_set['sum_punc_count'] = train_set['Summary'].apply(lambda x : len([a for a in x if a in string.punctuation]))
# the number of punctuations in the text
train_set['text_punc_count'] = train_set['Document'].apply(lambda x : len([a for a in x if a in string.punctuation]))